# 机器翻译与数据集
## 机器翻译
语言模型是自然语言处理的关键，而机器翻译是语言模型最成功的基准测试。因为机器翻译正是将输入序列转换成输出序列的*序列转换模型*（sequence transduction）的核心问题。序列转换模型在各类现代人工智能应用中发挥着至关重要的作用。本节将介绍机器翻译问题及其后文需要使用的数据集。

*机器翻译*（machine translation）指的是将序列从一种语言自动翻译成另一种语言。在使用神经网络进行端到端学习的兴起之前，统计学方法在这一领域一直占据主导地位。因为*统计机器翻译（statistical machine translation）涉及了翻译模型和语言模型等组成部分的统计分析，因此基于神经网络的方法通常被称为*神经机器翻译*（neuralmachine translation），用于将两种翻译模型区分开来。

本书的关注点是神经网络机器翻译方法，强调的是端到端的学习。与 :numref:`sec_language_model`中的语料库是单一语言的语言模型问题存在不同，机器翻译的数据集是由源语言和目标语言的文本序列对组成的。因此，我们需要一种完全不同的方法来预处理机器翻译数据集，而不是复用语言模型的预处理程序。下面，我们看一下如何将预处理后的数据加载到小批量中用于训练。

## 下载和预处理数据集
数据集下载地址：http://d2l-data.s3-accelerate.amazonaws.com/fra-eng.zip

组成的“英－法”数据集，数据集中的每一行都是制表符分隔的文本序列对，序列对由英文文本序列和翻译后的法语文本序列组成。请注意，每个文本序列可以是一个句子，也可以是包含多个句子的一个段落。在这个将英语翻译成法语的机器翻译问题中，英语是*源语言*（source language），法语是*目标语言*（target language）。


In [35]:
import os
import torch
from d2l import torch as d2l
import collections

In [36]:
def read_data_nmt():
    """载入英语-法语数据集"""
    data_dir = 'C:/Users/lenovo/pytorch-learning/data/fra-eng'
    with open(os.path.join(data_dir, 'fra.txt'), 'r',
             encoding='utf-8') as f:
        return f.read()

raw_text = read_data_nmt()

In [37]:
print(raw_text[:75])

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !



下载数据集后，原始文本数据需要经过几个预处理步骤。例如，我们用空格代替*不间断空格*（non-breaking space），使用小写字母替换大写字母，并在单词和标点符号之间插入空格。

In [38]:
def preprocess_nmt(text):
    """预处理“英语－法语”数据集"""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # 使用空格替换不间断空格
    # 使用小写字母替换大写字母
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # 在单词和标点符号之间插入空格
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)

In [39]:
print(text[:70])

go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	


## 词元化
和语言模型不一样，机器翻译中我们更常用单词级词元化。下面`tokenize_nmt`函数对前`num_examples`个文本序列对进行词元化，其中每个词元要么是一个词，要么是一个标点符号。此函数返回两个词元列表：`source`和`target`：`source[i]`是源语言（这里是英语）第$i$个文本序列的词元列表，`target[i]`是目标语言（这里是法语）第$i$个文本序列的词元列表。


In [40]:
def tokenize_nmt(text, num_examples=None):
    """词元化“英语－法语”数据数据集"""
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

In [41]:
source, target = tokenize_nmt(text)

In [42]:
source[:6], target[:6]

([['go', '.'],
  ['hi', '.'],
  ['run', '!'],
  ['run', '!'],
  ['who', '?'],
  ['wow', '!']],
 [['va', '!'],
  ['salut', '!'],
  ['cours', '!'],
  ['courez', '!'],
  ['qui', '?'],
  ['ça', 'alors', '!']])

## 词表
由于机器翻译数据集由语言对组成，因此我们可以分别为源语言和目标语言构建两个词表。使用单词级词元化时，词表大小将明显大于使用字符级词元化时的词表大小。为了缓解这一问题，这里我们将出现次数少于2次的低频率词元视为相同的未知（“&lt;unk&gt;”）词元。除此之外，我们还指定了额外的特定词元，例如在小批量时用于将序列填充到相同长度的填充词元（“&lt;pad&gt;”），以及序列的开始词元（“&lt;bos&gt;”）和结束词元（“&lt;eos&gt;”）。这些特殊词元在自然语言处理任务中比较常用。

In [43]:
class Vocab:
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens == None:
            tokens = []
        if reserved_tokens == None:
            reserved_tokens = []
        # 按出现频率排序
        counter = self.count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                 reverse=True)
        
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self.token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
                
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    # 统计词元的频率
    def count_corpus(self, tokens):
        tokens = [token for line in tokens for token in line]
        return collections.Counter(tokens)
    
    @property
    def unk(self):  # 未知词元的索引为0
        return 0

In [44]:
src_vocab = Vocab(source, min_freq=2,
                      reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab)

10012

In [45]:
tgt_vocab = Vocab(target, min_freq=2,
                 reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(tgt_vocab)

17851

## 加载数据集
回想一下，语言模型中的序列样本都有一个固定的长度，这个固定的长度是由参数`num_steps`指定的。在机器翻译中，每个样本都是由源和目标组成的文本序列对，其中的每个文本序列可能具有不同的长度。为了提高计算效率，我们可以通过截断和填充的方式实现一次处理一个小批量的文本序列。

假设同一个小批量中的每个序列都应该具有相同的长度`num_steps`，那么如果文本序列中的词元数目少于`num_steps`，我们将继续在其末尾添加特定的“&lt;pad&gt;”词元，直到其长度达到`num_steps`；反之，我们将截断文本序列，只取其前`num_steps`个词元，丢弃剩余的词元。这样，每个文本序列具有相同的长度，以便以相同形状的小批量进行加载。

如前所述，下面的`truncate_pad`函数用来截断或填充文本序列。

In [46]:
def truncate_pad(line, num_steps, padding_token):
    """截断或填充文本序列"""
    if len(line) > num_steps:
        return line[:num_steps]  # 截断
    return line + [padding_token] * (num_steps - len(line))  # 填充

In [47]:
truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])

[47, 4, 1, 1, 1, 1, 1, 1, 1, 1]

现在我们定义一个函数，可以将文本序列转换成小批量数据集用于训练。我们将特定的“&lt;eos&gt;”词元添加到所有序列的末尾，用于表示序列的结束。当模型通过一个词元接一个词元地生成序列进行预测时，生成的“&lt;eos&gt;”词元说明完成了序列输出工作。此外，我们还记录了每个文本序列的长度，统计长度时排除了填充词元，在稍后将要介绍的一些模型会需要这个长度信息。

In [48]:
def build_array_nmt(lines, vocab, num_steps):
    """将机器翻译的文本序列转换成小批量"""
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(
        l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    return array, valid_len

最后，我们定义`load_data_nmt`函数来返回数据迭代器，以及源语言和目标语言的两种词表。

In [49]:
def load_data_nmt(batch_size, num_steps, num_examples=600):
    """返回翻译数据集的迭代器和词表"""
    text = preprocess_nmt(read_data_nmt())
    source, target = tokenize_nmt(text, num_examples)
    src_vocab = Vocab(source, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = Vocab(target, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)
    data_array = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(*data_array),
                                            batch_size, shuffle=True)
    return data_iter, src_vocab, tgt_vocab

In [52]:
load_data_nmt_results = load_data_nmt(batch_size=2, num_steps=8)

In [56]:
torch.save(load_data_nmt_results, 'load_data_nmt_results.pt')

## 小结
* 机器翻译指的是将文本序列从一种语言自动翻译成另一种语言。
* 使用单词级词元化时的词表大小，将明显大于使用字符级词元化时的词表大小。为了缓解这一问题，我们可以将低频词元视为相同的未知词元。
* 通过截断和填充文本序列，可以保证所有的文本序列都具有相同的长度，以便以小批量的方式加载。